In [ ]:
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
import base64
import os
import shutil
import soundfile as sf
from natsort import natsorted
import textract
import textgrids
import json
from dtaidistance import dtw
import pitch_squeezer as ps
from tqdm import tqdm

In [ ]:
# list files in the folder containing all individual participant file
sujs = os.listdir("\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\RAW\\")

In [ ]:
sujs

In [ ]:
# select the subject you are going to work with
suj = 'subj01'

In [ ]:
path = "\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\RAW\\"+suj+".txt"

In [ ]:
# import data as bytes
text = textract.process(path)

In [ ]:
# split data by components 10 is training (should be 11 in your data as I added a component)
len(text.splitlines())

In [ ]:
# training data
trainingdata = json.loads(text.splitlines()[11].decode('utf-8'))

# Extract RAW DATA

In [ ]:
#####################################
# CONCATENATE DATA from all 4 blocks#
#####################################

block1data = json.loads(text.splitlines()[12].decode('utf-8'))
block2data = json.loads(text.splitlines()[13].decode('utf-8'))
block3data = json.loads(text.splitlines()[14].decode('utf-8'))
block4data = json.loads(text.splitlines()[15].decode('utf-8'))
allblocks = np.concatenate((block1data,block2data,block3data,block4data))

In [ ]:
##############################################
# create containers for each subtype of trial#
##############################################
RR1s =[]
GAPs = []
CONVs = []
RR2s = []

# retrive trials based on their id
trial_types = [['read&repeat1block1','read&repeat1block2','read&repeat1block3','read&repeat1block4'],
               ['gap_answerblock1','gap_answerblock2','gap_answerblock3','gap_answerblock4'],
               ['convergenceblock1','convergenceblock2','convergenceblock3','convergenceblock4'],
               ['read&repeat2block1','read&repeat2block2','read&repeat2block3','read&repeat2block4']]
trial_containers = [RR1s, GAPs,CONVs,RR2s]

for num,trial in enumerate(trial_types):
    for numi in range(len(allblocks)):
        try:
            if allblocks[numi]['trial_id'] in trial:
                trial_containers[num].append(allblocks[numi])
        except: 
            pass

In [ ]:
#############################################################
# make sure we have the desired number of trials per subtype#
#############################################################

assert(len(RR1s)==24) #6*4
assert(len(RR2s)==24) #6*4
assert(len(CONVs)==32) # 8*4

# /!\ if there is an error it means that the numbers chosen for the block trials are not the good ones
# it can happen when participants reload some components

In [ ]:
# read in presented stimuli properties you can look on the github repo 
stim_info = pd.read_csv("CONVERGENCE3\\STIMULI\\stimuli_AR.csv")
# create dictionnary to convert filtering properties to experimental conditions
conversion = {  '5.5_Inf': 'Normal_temporal',
                'Inf_0.35': 'Normal_spectral' ,  
                '23_Inf': 'Fast_temporal',
                'Inf_1.5': 'Fast_spectral'} 

In [ ]:
#########################################
# get names and contents for CONVERGENCE#
#########################################

names_conv = []
content_conv = []
stims_namesconv = []
conds2 = []
blocks2 = []
expcond = []
ars = []
shorts = []
for num in range(len(CONVs)):
    # get current condition
    curxcond = CONVs[num]['presented_stim'].split('/')[1].split('filter_')[1]
    # get current speed
    spee = conversion[curxcond].split('_')[0]
    # get AR from presented stim
    curar = stim_info.loc[(stim_info['sentences'] == CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0].lower())&(stim_info['speed'] == spee )]['articulation_rate'].values[0]
    names_conv.append('conv_'+str(num+1)+'.wav')
    content_conv.append(CONVs[num]['response'])
    stims_namesconv.append(CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0])
    conds2.append('conv')
    blocks2.append(CONVs[num]['trial_id'][-1])
    expcond.append(curxcond)
    ars.append(curar)
    shorts.append(stim_info.loc[(stim_info['sentences'] == CONVs[num]['stimulus'].split("25px\'>")[1].split("</p>")[0].lower())&(stim_info['speed'] == spee )]['short_names'].values[0])

dico2 = {
    'trial_type':conds2,
     'audios_names':names_conv,
    'audios_contents': content_conv,
    'stims_names': stims_namesconv,
    'blocks': blocks2,
    'conditions':expcond,
    'stim_AR': ars,
    'short_names':shorts}
df2 = pd.DataFrame.from_dict(dico2)


#################################
# get names and contents for RR1#
#################################

names_rr1 = []
content_rr1 = []
stims_names = []
conds = []
blocks = []
expcond2 = []
ars2 =[]
shorts2 = []
for num in range(len(RR1s)):
    curb = RR1s[num]['trial_id'][-1]
    names_rr1.append('rr1_'+str(num+1)+'.wav')
    content_rr1.append(RR1s[num]['response'])
    stims_names.append(RR1s[num]['stim_name'])
    conds.append('r&r1')
    blocks.append(curb)
    expcond2.append(df2.loc[df2["blocks"]==str(curb)]['conditions'].values[0])
    ars2.append(np.nan)
    shorts2.append(np.nan)

dico = {
    'trial_type':conds,
     'audios_names':names_rr1,
    'audios_contents': content_rr1,
    'stims_names': stims_names,
    'blocks':blocks,
    'conditions':expcond2,
    'stim_AR': ars2,
    'short_names':shorts2}

df1 = pd.DataFrame.from_dict(dico)

#################################
# get names and contents for RR2#
################################
names_rr2 = []
content_rr2 = []
stims_names3 = []
conds3 = []
blocks3 = []
expcond3 = []
ars3 = []
shorts3 = []
for num in range(len(RR2s)):
    curb2 = RR2s[num]['trial_id'][-1]
    names_rr2.append('rr2_'+str(num+1)+'.wav')
    content_rr2.append(RR2s[num]['response'])
    stims_names3.append(RR2s[num]['stim_name'])
    conds3.append('r&r2')
    blocks3.append(curb2)
    expcond3.append(df2.loc[df2["blocks"]==str(curb2)]['conditions'].values[0])
    ars3.append(np.nan)
    shorts3.append(np.nan)


dico3 = {
    'trial_type':conds3,
     'audios_names':names_rr2,
    'audios_contents': content_rr2,
    'stims_names': stims_names3,
    'blocks':blocks3,
    'conditions':expcond3,
    'stim_AR': ars3,
    'short_names':shorts3}

df3 = pd.DataFrame.from_dict(dico3)

#######################
# concatenate all data#
#######################
alldata = pd.concat((df1,df2,df3)).reset_index()

############################################################
# add experimental conditions based on filtering properties#
############################################################
speeds = []
filterings = []
for i in range(len(alldata)):
    curcond = alldata.iloc[i]['conditions']
    filterings.append(conversion[curcond].split('_')[1])
    speeds.append(conversion[curcond].split('_')[0])
alldata['speed'] = speeds
alldata['filtering'] = filterings


### Here you can modify the cell below to compute the performance per condition

In [ ]:
# get behavioral performance for gap in speech
gap_reps = [i['sound'].split('/')[1].split('.')[0] for i in GAPs]
gaps_conds = [i['stim_presented'].split('/')[1].split('/')[0] for i in GAPs]
perc_correct  = (np.sum([1 if i == 'good' else 0 for i in gap_reps])/len(gap_reps))*100
print('Subject performance on gap trials: '+str(np.round(perc_correct,2))+'%')

In [ ]:
# save individual dataframe
finaldat = alldata[['index', 'trial_type', 'audios_names','short_names', 'stims_names','blocks', 'conditions', 'stim_AR', 'speed', 'filtering']]


outpathOne = "CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\INFO\\"
if not os.path.exists(outpathOne):
    os.makedirs(outpathOne)  

finaldat.to_csv(outpathOne+suj+"_info.csv", index = None)


# Create audio files

In [ ]:
# create the folder for the audio files if it does not exist
audio_outpath = "CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\audio_responses\\"
if not os.path.exists(audio_outpath):
    os.makedirs(audio_outpath)  

In [ ]:
# aggregate all the names of the audio files
final_names = alldata['audios_names'].values
# aggregate all the audio files
final_audio = alldata['audios_contents'].values

In [ ]:
# Translate each succession of weird characters into .wav file
for i,(name,audio) in enumerate(zip(final_names,final_audio)):
    decodedData = base64.b64decode(audio)
    webmfile = audio_outpath+"temp.webm"
    with open(webmfile, 'wb') as file:
           file.write(decodedData)
    newtempath = str(webmfile).replace(os.path.sep, '/')
    newfinalfilepath = str(audio_outpath+name).replace(os.path.sep, '/')
    curcommand = 'ffmpeg -i '+newtempath+' ' + newfinalfilepath
    os.system(curcommand)
    os.remove(audio_outpath+"temp.webm") 


# Extract Pith and pitch contour

In [ ]:
# load indivudual participant info dataframe
curdata = pd.read_csv("CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\INFO\\"+suj+"_info.csv")

### Here you can have a look at sklearn to compute other metrics (e.g., cosine distance or euclidean distance)

In [ ]:
# sklearn --> https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html

# extract all features we need

yourmeanF0s = []
stimulimeanF0s = []
Distscontours = []


generalpath = "C:\\Users\\Jeremy\\Documents\\RESEARCH\\BEHAVIOR_2.0\\jatos_win_java\\study_assets_root\\CONVERGENCE3\\audio"
individualparticipantpath    = "C:\\Users\\Jeremy\\Desktop\\CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\subj23\\audio_responses\\"

for row in tqdm(range(len(curdata))):
    # determine condition
    curent_cond = curdata.iloc[row]['trial_type']    
    # retrieve audio path for my voice
    my_own = curdata.iloc[row]['audios_names']
    participantpath = individualparticipantpath+my_own
    
    if curent_cond == 'conv':
        # retieve audiopath for experimental stimuli
        presented = curdata.iloc[row]['short_names']
        cond = curdata.iloc[row]['conditions']    
        final_name = 'filter_' +cond+'_frm01_' + presented
        foldername = '\\filter_' +cond
        finalpathexpstim = foldername +'\\'+ final_name+".wav"
        exppath = generalpath+finalpathexpstim

    # here in the case where the trial type is 'conv' we compute both metrics (mean F0 and F0 contour) 
    # for both experimental audio stim and participants' productions
       # if no audio data (participants did not record something) just ad np.nan
        try: 
             # this for experiment stimuli
            #loading and computing FO from experimental stimuli
            f0, if0exp = ps.track_pitch(exppath, frame_rate=200)
            meanf0exp = np.mean(if0exp,0)

            # this for individual participant
            f0, if0 = ps.track_pitch(participantpath, frame_rate=200)
            meanf0participant = np.mean(if0,0)

            # Dynamic Time warping distance to compute distance between pitch contour
            F0distance = dtw.distance(if0exp, if0)
            # here you can also use another metric
            # for instance you can pad the shortest signal with zeros and compute euclidean distance or cosine distance
            Distscontours.append(F0distance)
            stimulimeanF0s.append(meanf0exp)
            yourmeanF0s.append(meanf0participant)

        except:
            f0, if0exp = ps.track_pitch(exppath, frame_rate=200)
            meanf0exp = np.mean(if0exp,0)
            stimulimeanF0s.append(meanf0exp)
            yourmeanF0s.append(np.nan)
            Distscontours.append(np.nan)
            
    # here in the case where the trial type is 'r&r1' or 'r&r2' we just compute un metric (mean F0) 
    # and only for participants' productions as the experimental stimuli were presented as written forms
    else:
        try: 
            # this for individual participant
            f0, if0 = ps.track_pitch(participantpath, frame_rate=200)
            meanf0participant = np.mean(if0,0)
            yourmeanF0s.append(meanf0participant)
            # no FO from experimental stimuli because written stimuli
            stimulimeanF0s.append(np.nan)
            # No distance because only one audio file
            Distscontours.append(np.nan)
        except:
            stimulimeanF0s.append(np.nan)
            yourmeanF0s.append(np.nan)
            Distscontours.append(np.nan)

In [ ]:
# Add the F0 stims and F0 stim pitch contour distance 
curdata['meanF0_stim'] = stimulimeanF0s
curdata['meanF0_subj'] = yourmeanF0s
curdata['pitchcontour_dist'] = Distscontours

In [ ]:
# keep only the convergence trial 
df3 = curdata.loc[curdata['trial_type']=='conv'].reset_index()

In [ ]:
# compute baseline F0
arcs = []
for i in range(len(df3)):
    c = df3.iloc[i]['conditions']
    arc = curdata.loc[(curdata['trial_type']=="r&r1")&(curdata['conditions']==c)]['meanF0_subj'].mean()
    arcs.append(arc)
# add the mean F0 to the dataframe
df3['baseline_F0_subj'] = arcs

In [ ]:
# abs of difference between stim F0 and subject mean F0
df3['absdiff'] = abs(df3['meanF0_stim'] - df3['meanF0_subj'])
df3['subj'] = np.repeat(suj,len(df3))

In [ ]:
df3.to_csv("CONVERGENCE3_DATA_ANALYSIS\\data_prolific\\INDIVIDUAL\\"+suj+"\\INFO\\"+suj+"_infoF0.csv"))